In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.10.1
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from config import *
import os

print("BASE_DIR:", BASE_DIR)
print("DATA_PATH:", DATA_PATH)
print("File exists:", os.path.exists(DATA_PATH))


BASE_DIR: C:\Users\AnweshaSingh\anaconda_projects\CEP_microservice
DATA_PATH: C:\Users\AnweshaSingh\anaconda_projects\CEP_microservice\data\EXCELdata_center_carbon_emission_dataset_75000.xlsx
File exists: True


In [3]:
from config import *
from utils import *
from data_preprocessing import *
from models import *
from train import *
from evaluate import *

set_seeds(RANDOM_SEED)

import os

print("DATA_PATH:", DATA_PATH)
print("File exists:", os.path.exists(DATA_PATH))


df = load_and_prepare_data(DATA_PATH)
train, val, test = split_data(df)

# ================= ENCODE CATEGORICAL FEATURES =================

categorical_cols = []

# Detect object (string) columns automatically
for col in train.columns:
    if train[col].dtype == 'object' and col != 'timestamp':
        categorical_cols.append(col)

print("Categorical columns detected:", categorical_cols)

# One-hot encode
train = pd.get_dummies(train, columns=categorical_cols, drop_first=True)
val = pd.get_dummies(val, columns=categorical_cols, drop_first=True)
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True)

# ================= ALIGN COLUMNS ACROSS SPLITS =================

# Align train and validation
train, val = train.align(val, join='left', axis=1, fill_value=0)

# Align train and test
train, test = train.align(test, join='left', axis=1, fill_value=0)

print("Encoding and column alignment complete.")



feature_cols = [
    col for col in train.columns
    if col not in ['timestamp', TARGET_COLUMN]
]




DATA_PATH: C:\Users\AnweshaSingh\anaconda_projects\CEP_microservice\data\EXCELdata_center_carbon_emission_dataset_75000.xlsx
File exists: True
Categorical columns detected: ['MAC']
Encoding and column alignment complete.


In [4]:
train.dtypes


weekday                           int64
timestamp                datetime64[ns]
voltage                         float64
current                         float64
power                           float64
frequency                       float64
energy                          float64
power_factor                    float64
esp32_temperature               float64
cpu_temperature                 float64
gpu_temperature                 float64
cpu_usage_percent               float64
cpu_power_watts                 float64
gpu_usage_percent               float64
gpu_power_watts                 float64
ram_usage_percent               float64
ram_power_watts                 float64
hour                              int32
dayofweek                         int32
voltage_lag1                    float64
MAC_4D:62:06:13:97:31              bool
MAC_5E:63:07:14:98:32              bool
MAC_6F:64:08:15:99:33              bool
dtype: object

In [ ]:
X_train_seq, y_train_seq = create_sequences(
    train[feature_cols].values,
    train[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

X_val_seq, y_val_seq = create_sequences(
    val[feature_cols].values,
    val[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

hyper_bounds = {
    'lstm_layers': ([2, 3], 'int'),
    'units': ([64, 128], 'categorical'),
    'dropout': ([0.3, 0.6], 'float'),
    'optimizer': (['adamw'], 'categorical'),
    'learning_rate': ([5e-5, 5e-4], 'float_log'),
    'batch_size': ([32], 'categorical')
}

optimizer = NinjaOptimizationAlgorithm(
    objective_function=lambda p: objective_function_lstm(
        p, X_train_seq, y_train_seq, X_val_seq, y_val_seq
    ),
    bounds=hyper_bounds,
    n_agents=N_AGENTS,
    max_iterations=MAX_ITERATIONS,
    exploration_factor=EXPLORATION_FACTOR,
    exploitation_factor=EXPLOITATION_FACTOR
)

best_params, best_loss, convergence = optimizer.optimize()


Iteration 1: Best Loss = 6711345.500000


In [ ]:
from evaluateresults import evaluate_regression_model, plot_predictions

# ================= FINAL TEST SET EVALUATION =================
test_metrics = evaluate_regression_model(
    final_model,
    X_test_seq,
    y_test_seq,
    dataset_name="Test"
)

# Optional: qualitative visualisation
y_test_pred = final_model.predict(X_test_seq).flatten()
plot_predictions(
    y_test_seq,
    y_test_pred,
    save_path="results/test_predictions.png"
)
